In [9]:
#import libraries
import pandas as pd
import numpy as np
import cv2

In [10]:
#import pre-calculared cluster centers, our dictionary words
import pickle
pkl_file = open("WORDS_kcen.pkl", 'rb')
WORDS_kcen = pickle.load(pkl_file)
pkl_file.close()

In [11]:
#create the dictionary knn model
from sklearn.neighbors import KNeighborsClassifier
word_detect = KNeighborsClassifier(n_neighbors=1)
y = np.arange(len(WORDS_kcen))
word_detect.fit(WORDS_kcen,y)

KNeighborsClassifier(n_neighbors=1)

In [16]:
image_paths_TRAIN = {"aeroplane":"VOCdevkit/VOC2012/ImageSets/Main/aeroplane_train.txt","bird":"VOCdevkit/VOC2012/ImageSets/Main/bird_train.txt","person":"VOCdevkit/VOC2012/ImageSets/Main/person_train.txt","sofa":"VOCdevkit/VOC2012/ImageSets/Main/sofa_train.txt"}
image_paths_TEST = {"aeroplane":"VOCdevkit/VOC2012/ImageSets/Main/aeroplane_trainval.txt","bird":"VOCdevkit/VOC2012/ImageSets/Main/bird_trainval.txt","person":"VOCdevkit/VOC2012/ImageSets/Main/person_trainval.txt","sofa":"VOCdevkit/VOC2012/ImageSets/Main/sofa_trainval.txt"}
classes = ["person","bird","aeroplane","sofa"]
im_folder = "VOCdevkit/VOC2012/JPEGImages/"
sift = cv2.SIFT_create(nfeatures=100) #sift detector object

In [13]:
#we'll use this function to feature extract both the test and train dataset.

def feature_extract_bulk(image_paths):
    #create pandas dataframe to store the curated features extracted from our images
    cols  = list(np.arange(len(WORDS_kcen)))
    #insert column names
    cols.insert(0,"file") 
    cols.insert(1,"class")
    df = pd.DataFrame(columns=cols)
    
    for i_cl, clas in enumerate(classes):
        with open(image_paths[clas]) as im_paths:
            print("Class: ",im_paths) #for progress tracking purposes
            paths = im_paths.readlines()  #read all paths in the paths file
            for i,path in enumerate(paths): #iterate all images 
                image = cv2.imread(im_folder+paths[i][:-4]+".jpg") #read image
                image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) #convert to grayscale
                kp, des = sift.detectAndCompute(image,None)
                try: #some images for some reason, the sift detector outputs None
                    pred = word_detect.predict(des)
                    row = dict(pd.Series(pred).value_counts())
                    row["file"] = paths[i][:-4] #add file name
                    row["class"] = i_cl #add class value
                    df.loc[len(df)] = row #add row to dataframe
                except:
                    pass
    return df

In [14]:
df_train = feature_extract_bulk(image_paths_TRAIN)

Class:  <_io.TextIOWrapper name='VOCdevkit/VOC2012/ImageSets/Main/person_train.txt' mode='r' encoding='UTF-8'>
Class:  <_io.TextIOWrapper name='VOCdevkit/VOC2012/ImageSets/Main/bird_train.txt' mode='r' encoding='UTF-8'>
Class:  <_io.TextIOWrapper name='VOCdevkit/VOC2012/ImageSets/Main/aeroplane_train.txt' mode='r' encoding='UTF-8'>
Class:  <_io.TextIOWrapper name='VOCdevkit/VOC2012/ImageSets/Main/sofa_train.txt' mode='r' encoding='UTF-8'>


In [17]:
df_test = feature_extract_bulk(image_paths_TEST)

Class:  <_io.TextIOWrapper name='VOCdevkit/VOC2012/ImageSets/Main/person_trainval.txt' mode='r' encoding='UTF-8'>
Class:  <_io.TextIOWrapper name='VOCdevkit/VOC2012/ImageSets/Main/bird_trainval.txt' mode='r' encoding='UTF-8'>
Class:  <_io.TextIOWrapper name='VOCdevkit/VOC2012/ImageSets/Main/aeroplane_trainval.txt' mode='r' encoding='UTF-8'>
Class:  <_io.TextIOWrapper name='VOCdevkit/VOC2012/ImageSets/Main/sofa_trainval.txt' mode='r' encoding='UTF-8'>


In [23]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [25]:
df_train.to_csv("df_train.csv")
df_test.to_csv("df_test.csv")

In [24]:
from sklearn import svm
clf = svm.SVC()
clf.fit(df_train.drop(['file',"class"], axis=1),df_train["class"])

SVC()

In [26]:
clf.score(df_test.drop(['file',"class"], axis=1),df_test["class"])

0.25

In [28]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(df_train.drop(['file',"class"], axis=1),df_train["class"])

KNeighborsClassifier(n_neighbors=3)

In [29]:
neigh.score(df_test.drop(['file',"class"], axis=1),df_test["class"])